<a href="https://colab.research.google.com/github/jainnipun/MachineLearning/blob/master/Clustering/CustomerSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
#files.upload()

In [0]:

#Install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#!kaggle datasets list

In [0]:
#!kaggle competitions download -c microsoft-malware-prediction

In [0]:
#!kaggle datasets download -d lava18/google-play-store-apps

In [13]:
!kaggle datasets download -d carrie1/ecommerce-data

 63% 5.00M/7.95M [00:00<00:00, 36.5MB/s]
100% 7.95M/7.95M [00:00<00:00, 50.6MB/s]


In [14]:
!unzip -d ecommerce ecommerce-data.zip

Archive:  ecommerce-data.zip
  inflating: ecommerce/data.csv      


In [15]:
!ls -a

.  ..  .config	ecommerce  ecommerce-data.zip  kaggle.json  sample_data


In [16]:
!ls ecommerce

data.csv


In [0]:
import pandas as pd
data=pd.read_csv('ecommerce/data.csv',encoding = "ISO-8859-1")

In [0]:
import datetime

data['InvoiceDate'] = data['InvoiceDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%Y-%m-%d"))

In [19]:
print(data['InvoiceDate'].min())
print(data['InvoiceDate'].max())
max_date = data['InvoiceDate'].max()

2010-12-01
2011-12-09


In [0]:
data['Amount'] = data['Quantity'] * data['UnitPrice']

In [0]:
data=data.loc[data['CustomerID'].isnull()==False]

In [0]:
#data.loc[data['Amount']==0]

In [0]:
#Removing records where Quatity is 0

data = data.loc[data['Quantity']!=0]

In [0]:
#data.loc[data['CustomerID']==18274.0]
#data.loc[data['CustomerID'] == 18276.0]


In [0]:
#data.loc[data['CustomerID'] == 12347.0]


In [24]:
#Viewing nan records - if any present in data we will rectify that

nans = lambda df: df[df.isnull().any(axis=1)]
nans(data)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount


In [0]:
#Not remove data where amount is 0 as we will consider free purchases as well
#data = data[data.Amount != 0]

In [0]:
#processedData = data.groupby(by=['CustomerID'])

In [0]:
# Recency  - Last Transaction date
# Frequency - Total Tranactions
# Monetary - Total Spendings

#processedData = pd.DataFrame(columns=['CustomerId', 'Recency', 'Frequency','Monetary'])
processedData = pd.DataFrame(columns=[ 'Recency', 'Frequency','Monetary'])

In [0]:
data_group_by_customers = data.groupby(by=['CustomerID'])

In [0]:
#data_group_by_customers.InvoiceNo.nunique()

In [0]:
#Counting unique invoice per customer
processedData['Frequency'] = data.groupby(by=['CustomerID']).InvoiceNo.nunique()

#Fetching last invoice date per customer
processedData['Recency']=data.groupby(by='CustomerID').max()['InvoiceDate']

#Fetching total spending per customer
processedData['Monetary'] =  data.groupby(by='CustomerID').sum()[['Amount']]

#processedData['CustomerId']=(list(processedData.index.values)) 

In [0]:
#Removing negative records - where refund greater than purchase
processedData = processedData.loc[processedData['Monetary']>=0.0]


In [0]:
import datetime

now = datetime.datetime.now()
dataset_max_date = datetime.datetime.strptime(max_date,'%Y-%m-%d')

processedData['Recency']=processedData['Recency'].apply(lambda x:(dataset_max_date-datetime.datetime.strptime(x,'%Y-%m-%d')).days)


In [56]:
processedData

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,325,2,0.000000e+00
12347.0,2,7,4.310000e+03
12348.0,75,4,1.797240e+03
12349.0,18,1,1.757550e+03
12350.0,310,1,3.344000e+02
12352.0,36,11,1.545410e+03
12353.0,204,1,8.900000e+01
12354.0,232,1,1.079400e+03
12355.0,214,1,4.594000e+02


In [59]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

range_n_clusters = [2, 3, 4, 5, 6,7,8,9,10]

for k in range_n_clusters:
   
    # Run the Kmeans algorithm
    km = KMeans(n_clusters=k)
    cluster_labels = km.fit_predict(processedData)
    centroids = km.cluster_centers_

    silhouette_avg = silhouette_score(processedData, cluster_labels)
    
    # Get silhouette samples
    silhouette_vals = silhouette_samples(processedData, cluster_labels)
    
    print(k,silhouette_avg)


2 0.9862953974173035
3 0.9630003427648444
4 0.8841566942953814
5 0.8158505063445289
6 0.772658227161448
7 0.7726653987752443
8 0.7232620351607983
9 0.6779532755212787
10 0.6774173649697526
